### General imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import cv2 
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from scipy.signal import convolve2d as conv2

### Auxiliary functions

In [ ]:
def render_images(images):
    """Renders images
    
    Args:
        images: dictionary with images
    """
    
    # TODO: Develop function to render images. Make sure to have a descriptive caption for each image
    for img_name, imgs in images.items():
        # plt.figure()
        plt.imshow(imgs)
        plt.title(img_name)
        plt.show()
    
    pass

In [ ]:
def load_images(names, render=True):
    """Loads images from files and renders them, if necessary

    Args:
        names: list of image files to load
        render: flag that defines if image should be rendered

    Returns:
        Dictionary with loaded images
    """

    # TODO: Develop function to load images from files
    img_dic = {}
    
    for names_list in names:
        img_read = Image.open(names_list)
        img_read = img_read.convert('1')
        img_dic[names_list] = img_read
        
    images = render_images(img_dic)

    return images

## 1 Fourier filtering

### 1.1 Preparation of dataset

In [ ]:
def make_pattern(dx, theta, shape):
    """Generates image with periodic structure (`stripes`)

    Args:
        dx: stripe thickness
        theta: angle between stripes and x-axis
        shape: shape of resulting image

    Returns:
        Image with required pattern as numpy array
    """

    # TODO: Develop function to generate image with required periodic structure
    
    # Generate empty matrix & line strips
    gen_mat = np.zeros((shape,shape), np.uint8)
    lines = np.linspace(0, shape, shape//dx, dtype=int)
    
    for i in range(0, len(lines)-1, 2):
        gen_mat[:, lines[i]:lines[i+1]] = 255
    
    pad = 200
    img = np.pad(gen_mat, 100, mode='constant')
    img = np.array(Image.fromarray(img).rotate(theta))
    image = img[pad:-pad, pad:-pad]

    return image

In [ ]:
# TODO: Show generated images with periodic structure
# Total: 4 images
# 1st 
Image.fromarray(make_pattern(10, 15, 1000))

In [ ]:
# 2nd
Image.fromarray(make_pattern(25, 30, 1000))

In [ ]:
# 3rd
Image.fromarray(make_pattern(50, 45, 1000))

In [ ]:
# 4th
Image.fromarray(make_pattern(100, 90, 1000))

In [ ]:
# TODO: Save generated images with periodic structure on harddrive
# Total: a dozen of images

dx = [10, 25, 50, 100, 5, 10, 15, 20, 30, 35, 40, 45]
theta = [15, 30, 45, 90, 125, 145, 180, 250, 270, 320, 60, 75]

for i in range(len(dx)):
    for j in range(len(theta)):
        img = Image.fromarray(make_pattern(dx[i], theta[j], 1000))
        img.save(f'data_{i+1}.png')

In [ ]:
# TODO: Load and render captured photos
# Total: 4 images

load_images(['data_1.png', 'data_2.png', 'data_3.png', 'data_4.png']);

### 1.2 Processing of acquired dataset

In [ ]:
def fft_image(image):
    """Calculates Fourier Transform image

    Args:
        image: image for which Fourier Transform image is calculated

    Returns:
        Fourier Transform image
    """

    # TODO: Develop function to calculate Fourier Transform image
    
    # Reading an image in default mode
    image = plt.imread(image)
    image = image[:, :, :3].mean(axis=2)  # Convert to grayscale
    plt.set_cmap("gray")
    ft = np.fft.ifftshift(image)
    ft = np.fft.fft2(ft)
    result = np.fft.fftshift(ft)

    return result

In [ ]:
# TODO: Calculate Fourier Transform image of captured photos
# Total: 4 Fourier Transform images

print(fft_image('data/DSC_0229.JPG'))
Image.open('data/DSC_0229.JPG')

In [ ]:
print(fft_image('data/DSC_0230.JPG'))
Image.open('data/DSC_0230.JPG')

In [ ]:
print(fft_image('data/DSC_0231.JPG'))
Image.open('data/DSC_0231.JPG')

In [ ]:
print(fft_image('data/DSC_0232.JPG'))
Image.open('data/DSC_0232.JPG')

In [ ]:
# TODO: Display Fourier Transform image of captured photos
# Total: 4 images
ft = fft_image('data/DSC_0229.JPG')
plt.imshow(np.log(abs(ft)))
plt.axis("off")
plt.show()

In [ ]:
ft = fft_image('data/DSC_0230.JPG')
plt.imshow(np.log(abs(ft)))
plt.axis("off")
plt.show()

In [ ]:
ft = fft_image('data/DSC_0231.JPG')
plt.imshow(np.log(abs(ft)))
plt.axis("off")
plt.show()

In [ ]:
ft = fft_image('data/DSC_0232.JPG')
plt.imshow(np.log(abs(ft)))
plt.axis("off")
plt.show()

In [ ]:
# Differenct delta x and theta leads to the different brightness of fourier transform pictures.

*`TODO`*: Qualitatively describe your observations of the changes to the Fourier spectrum as a function of ∆x and θ

In [ ]:
def fouirer_filter(source):
    """Filters image to get rid of `striped` illumination pattern

    Args:
        source: original image with hardware overlayed `striped` illumination pattern
        theta: angle between stripes and x-axis
        shape: shape of resulting image

    Returns:
        Filtered image
    """

    # TODO: Develop function to filter image
    image = plt.imread(source)
    image = image[:, :, :3].mean(axis=2)  
    plt.set_cmap("gray")
    ft = np.fft.fft2(image)
    fft = np.fft.fftshift(ft)
    
    # Try to change this!!!    
    pad = 16
    
    x_cut = fft.shape[0]//2 - pad, fft.shape[0]//2 + pad
    y_cut = fft.shape[1]//2 - pad, fft.shape[1]//2
    chunk = fft[x_cut[0]:x_cut[1], y_cut[0]:y_cut[1]]
    
    fft[x_cut[0]:x_cut[1], y_cut[0]:y_cut[1]] = 0
    
    x_cut = fft.shape[0]//2 - pad, fft.shape[0]//2 + pad
    y_cut = fft.shape[1]//2+2, fft.shape[1]//2 + pad
    chunk = fft[x_cut[0]:x_cut[1], y_cut[0]:y_cut[1]]
    fft[x_cut[0]:x_cut[1], y_cut[0]:y_cut[1]] = 0
       
    fourier = np.fft.ifft2(np.fft.ifftshift(fft))
    image = np.real(fourier)

    return image

In [ ]:
# TODO: Test fouirer_filter() using captured photos
# Total: Render 4 filtered images

remove_1 = fouirer_filter('DSC_0229.JPG')
plt.imshow(remove_1, cmap = 'gray')

In [ ]:
remove_2 = fouirer_filter('DSC_0230.JPG')
plt.imshow(remove_2, cmap = 'gray')

In [ ]:
remove_3 = fouirer_filter('DSC_0231.JPG')
plt.imshow(remove_3, cmap = 'gray')

In [ ]:
remove_4 = fouirer_filter('DSC_0232.JPG')
plt.imshow(remove_4, cmap = 'gray')

In [ ]:
# TODO: Compare filtered images to thier originals using SSIM score
# Total: 4 comparisons

ref_1 = plt.imread('DSC_0229.JPG')
ref_1 = ref_1[:, :, :3].mean(axis=2)

ref_2 = plt.imread('DSC_0230.JPG')
ref_2 = ref_2[:, :, :3].mean(axis=2)

ref_3 = plt.imread('DSC_0231.JPG')
ref_3 = ref_3[:, :, :3].mean(axis=2)

ref_4 = plt.imread('DSC_0232.JPG')
ref_4 = ref_4[:, :, :3].mean(axis=2)

ssim_1 = ssim(ref_1, remove_1)
ssim_2 = ssim(ref_2, remove_2)
ssim_3 = ssim(ref_3, remove_3)
ssim_4 = ssim(ref_4, remove_4)

print('{0:<8}({1:<12}| {2:<12}): {3:.8f}'.format('SSIM', 'Original 1', 'Filtered 1', ssim_1))
print('{0:<8}({1:<12}| {2:<12}): {3:.8f}'.format('SSIM', 'Original 2', 'Filtered 2', ssim_2))
print('{0:<8}({1:<12}| {2:<12}): {3:.8f}'.format('SSIM', 'Original 3', 'Filtered 3', ssim_3))
print('{0:<8}({1:<12}| {2:<12}): {3:.8f}'.format('SSIM', 'Original 4', 'Filtered 4', ssim_4))

*`TODO`*: Qualitatively describe your observations and suggest a concept to eliminate the artifacts in the filtered images.

Ans: I found that the method that I use (remove only the center of FT) is not good enough. It would be great if  can apply other removal function (e.g. sine) or remove other part of FT because the source code pattern has other parameter (e.g. theta). The result should be better but loose sharpness.



In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 5), sharex=True, sharey=True)

plt.gray()

fft_pic = Image.open("fft_pic.png")
fft_pic = fft_pic.convert('1')

ax[0].imshow(remove_4, cmap = 'gray')
ax[0].axis('off')
ax[0].set_title('1st_remove')

ax[1].imshow(fft_pic)
ax[1].axis('off')
ax[1].set_title('recovered from Image J')

## 2 Bokeh deconvolution

### 2.1 Image recovery from software-originated Bokeh effect

In [ ]:
# TODO: Load photo of decoration lights
# Total: 1 image

Image.open('DSC_0421.JPG')

In [ ]:
def make_synthetic_psf(shape):
    """Generates synthetic PSF of specified shape

    Args:
        shape: shape of synthetic PSF

    Returns:
        Synthetic PSF as numpy array
    """

    # TODO: Place code for synthetic PSF generation  into specific function
    psf =  np.array([[0, 0, 1, 0, 0], 
                     [0, 1, 1, 1, 0],
                     [1, 1, 1, 1, 1],
                     [0, 1, 1, 1, 0],
                     [0, 0, 1, 0, 0]], dtype="float64")/shape

    return psf

In [ ]:
# TODO: Calculate convolution of original photo of decoration lights and synthetic PSF
# TODO: Bokeh effect should be perceptible
# Total: 1 image

from skimage import restoration

bokeh_1 = Image.open("DSC_0421.JPG")
bokeh = bokeh_1.convert('1')
psf = make_synthetic_psf(200)
convo = conv2(bokeh, psf, 'same')

rng = np.random.default_rng()
convo += 0.1 * convo.std() * rng.standard_normal(convo.shape)

plt.imshow(convo, cmap = 'gray')

In [ ]:
# TODO: Recover the original image from the convolved image
# Total: 1 image

deconvolved, _ = restoration.unsupervised_wiener(convo, psf)
plt.imshow(deconvolved)

In [ ]:
# TODO: Display the original image, the convolved Bokeh image, and the recovered one side-by-side
# Total: a row of 3 images

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 5), sharex=True, sharey=True)

plt.gray()

orig = Image.open("DSC_0421.JPG")
orig = orig.convert('1')

ax[0].imshow(orig)
ax[0].axis('off')
ax[0].set_title('original')

ax[1].imshow(convo, cmap = 'gray')
ax[1].axis('off')
ax[1].set_title('convolved')

ax[2].imshow(deconvolved)
ax[2].axis('off')
ax[2].set_title('recovered')

fig.tight_layout()

plt.show()

*`TODO`*: Describe what the custom shape of the PSF did to the original image and what are the differences in the recovered and the original photos

Ans: The custom shape of psf is rhombus and when we apply to the original image, the picture is brightened. The difference between recovered and original images is sharpness. Recovered image leads to the loss in sharpness.

### 2.2 Image recovery from hardware-originated Bokeh effect

In [ ]:
# TODO: Load and render the captured photo of decoration light with perceptible Bokeh effect (hardware-based)
# Total: 1 image
Image.open('DSC_0422.JPG')

In [ ]:
def recover_from_hw_Bokeh(image, psf):
    """Recovers clean image from image with hardware-originated Bokeh effect

    Args:
        image: image with Bokeh-effect
        psf: PSF of known shape (may required scaling or other adjustment)

    Returns:
        Clean image
    """
    
    # TODO: Place code for recovery of the clean image of the decoration lights from the captured image with Bokeh effect
    
    return recovered_image

#### Using imageJ ####

I find FT and select only the white circle inside rhombus. The result will be shown in the next cell (loss sharpness).

In [ ]:
# TODO: Display the clean image side-by-side with original photo of decoration lights
# Total: a row of 2 images
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 5), sharex=True, sharey=True)

plt.gray()

bok_rec_ref = Image.open("DSC_0422.JPG")
bok_rec_ref = bok_rec_ref.convert('1')

bok_rec_rec = Image.open("recovered_1.png")
bok_rec_rec = bok_rec_rec.convert('1')


ax[0].imshow(bok_rec_ref)
ax[0].axis('off')
ax[0].set_title('original')

ax[1].imshow(bok_rec_rec)
ax[1].axis('off')
ax[1].set_title('recovered')